In [ ]:
# Setting up the environment

import svelte_widget
import xgboost
import shap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Upload and set up data

from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

df = pd.read_csv("BRCA.csv")

# Deal with non-numerical variables

df = df.drop(df.columns[0], axis=1)
df['Gender'] = df['Gender'].map({"FEMALE": 0, "MALE": 1})
df['Tumour_Stage'] = df['Tumour_Stage'].map({"I": 0, "II": 1, "III": 2})
df['Histology'] = df['Histology'].map({"Infiltrating Ductal Carcinoma": 0, "Mucinous Carcinoma": 1})
df['ER status'] = df['ER status'].map({"Negative": 0, "Positive": 1})
df['PR status'] = df['PR status'].map({'Negative': 0, 'Positive': 1})
df['HER2 status'] = df['HER2 status'].map({"Negative": 0, "Positive": 1})
df['Surgery_type'] = df['Surgery_type'].map({"Modified Radical Mastectomy": 0, "Lumpectomy": 1, "Simple Mastectomy": 2})
df['Patient_Status'] = df['Patient_Status'].map({"Dead": 0, "Alive": 1})
# Temporal variables for now are not considered
'''
for i in range(0, df.shape[0]):
    df.loc[i, 'Date_of_Surgery'] = datetime.strptime(df.loc[i, 'Date_of_Surgery'], '%d-%b-%y')
    df.loc[i, 'Date_of_Last_Visit'] = datetime.strptime(df.loc[i, 'Date_of_Last_Visit'], '%d-%b-%y')
'''
df = df.drop(['Date_of_Surgery', 'Date_of_Last_Visit'], axis=1)

df = df.dropna()

# Initialize the scaler
scaler = MinMaxScaler()

# Apply the scaler to every column in the dataframe
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

X = df.drop(columns=['Patient_Status'])

# We do not want to scale Y, so widget will display real values
# for predictions and not scaled ones
y = df['Patient_Status']

In [ ]:
# Defining and training two Neural Networks

from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

import tensorflow as tf
from tensorflow import keras

# Define the first neural network model
model_1 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_1.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Define the second neural network model
model_2 = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_2.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
z = svelte_widget.ExampleWidget(X_test,y_test,model_1.predict,model_2.predict)
z